ID Number: 33385806

# EEG-based BCI using Emotiv Epoc X and Visual Imagery: An Exploratory Study

## MSc Project for Computational Cognitive Neuroscience 2020/2021

### EEG Pre-Processing Analysis

1. Import libraries
2. Load dataset
3. Remove unwanted channels
4. Bandpass filter the data
5. Create Epochs
6. Artefact Correction with Independent Compontent Analysis (ICA)
7. Manual Bad epochs rejections
8. Save dataset

### 1. Import libraries

In [ ]:
%%capture libraries    

import sys
import os
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install mne
!{sys.executable} -m pip install mne-features
import numpy as np
import matplotlib 
import pathlib
import mne
import seaborn as sns
import pandas as pd
from mne.io import concatenate_raws, read_raw_edf
from mne import Epochs, create_info, events_from_annotations
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs,corrmap
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch, tfr_stockwell,tfr_multitaper,tfr_array_morlet,AverageTFR
from scipy import signal
from scipy.integrate import simps
matplotlib.use('Qt5Agg') #allow interactive plots
import matplotlib.pyplot as plt

%run EEG_functions_1.ipynb import load_data, excl_chan, filter_data, make_epochs, plot_data, epochs_power #upload the notebook with the python functions
 

### 2. Load the raw EEG dataset

In [ ]:
raw_datasets = load_data(os.getcwd()); #30 sessions in .edf format will be uploaded

### 3. Exclude unwanted channels

In [ ]:
include_channels = ['AF3','F7','F3','FC5','T7','P7','O1','O2','P8','T8','FC6','F4','F8','AF4']; #reference_channels = ['CQ_CMS', 'CQ_DRL']
excl_chan(raw_datasets) #remove the channels not included in the above list

#### Plot the unfiltered power spectrum density (PSD) from one session:

In [ ]:
raw_datasets[1].plot_psd(average=True)

### 4. Apply Band-Pass Filter between 1-30Hz

In [ ]:
filter_data(raw_datasets)

In [ ]:
#Explore the power spectrum of the filtered dataset, to check if the power line noise has been filtered out
raw_datasets[1].plot_psd(average=True)

### 5. Create Epochs 

Each epoch has a duration of 9.5 seconds. The first and last 250 milliseconds have been removed to avoid overlap between the events. 

In [ ]:
epoched_data=make_epochs(raw_datasets, 10) 
print(epoched_data.get_data().shape) #the final shape is n_epochs, chans, samples 

In [ ]:
print(epoched_data)

### 6. Apply ICA to the epoched data

In [ ]:
picks = raw_datasets[0].info['ch_names'] #define the eeg channels we want to include in the analysis, in this case all of them
ica=ICA(n_components=14, method='fastica', max_iter=10000, random_state=89) #define the parameters
ica.fit(epoched_data,  picks = picks, reject = dict(eeg = 200e-6)) #apply ICA to epochs

#### Plot ICA components

In [ ]:
ica.plot_components(picks=range(14), inst=epoched_data)   

#### Identify the components to exclude:

In [ ]:
ica.exclude=[0,1,13] #exclude eye movements, heartbeat and saccade

#### Before excluding the above identified ICA components, check also their time courses:

In [ ]:
ica.plot_sources(epoched_data)

#### It seems that also ICA components 7 and 8 might be problematic, use plot_overlay to overlay the raw and cleaned signals and see if by exlcuding these further two components there is a substantial difference:


In [ ]:
epoevo=epoched_data.average() #create an evoked object
ica.plot_overlay(epoevo, exclude=[0,1,13], picks='eeg')  

In [ ]:
ica.plot_overlay(epoevo, exclude=[0,1,13,7,8], picks='eeg')  

#### The signal didn't change after the removal of components 7 and 8. 
#### Exclude the components earlier identified:

In [ ]:
ica.apply(epoched_data, exclude=ica.exclude)  

#### 7. Reject Bad Epochs Manually, after visual inspections

#### A total of 20 epochs have been rejected.

In [ ]:
epochs.save('epoched_data_280-epo.fif', overwrite=True) #how to save the cleaned epochs after manual rejections